### This files generate bundles files to for ParaSweepers, currently standalone. the specific script below is to define model options for GEM-2 
especially after the overwrite - initial SEI thickness and neg porosity,
 based on initial capacity loss and partial molar volume of SEI

In [1]:
# magic lines that avoid re-start 
%load_ext autoreload
%autoreload 2
import csv, random, os
import pybamm as pb;import pandas as pd;import numpy as np;
import os, json,openpyxl,traceback,multiprocessing,scipy.optimize
import matplotlib.pyplot as plt;
import imageio,timeit,random,time, signal
from scipy.io import savemat,loadmat;
from pybamm import constants,exp;import matplotlib as mpl; 
fs=17; 
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

In [2]:
parameter_names = [
    "Scan No",
    "Exp No.", # 2,3,5
    "Ageing temperature", # 10,25,40
    "Mesh list",  
    'Inner SEI lithium interstitial diffusivity [m2.s-1]',#1e-19~5e-18
    'Dead lithium decay constant [s-1]',
    'Lithium plating kinetic rate constant [m.s-1]',
    'Negative electrode LAM constant proportional term [s-1]',
    'Positive electrode LAM constant proportional term [s-1]',
    'Negative electrode cracking rate',
    'Outer SEI partial molar volume [m3.mol-1]',
    "SEI growth activation energy [J.mol-1]", # 1e4~3.8e4
    "Negative cracking growth activation energy [J.mol-1]",#0
    "Negative electrode diffusivity activation energy [J.mol-1]",#1.7e4
    "Positive electrode diffusivity activation energy [J.mol-1]",#1.2e4
    #"Contact resistance [Ohm]",#0.010
    #'Total heat transfer coefficient [W.m-2.K-1]',#20
    'Initial electrolyte excessive amount ratio', # 1.0 or 0.99
]
para_short_name = [
    "Exp", # 2,3,5
    "T", # 10,25,40
    "Mesh",
    'Dint',#1e-19~5e-18
    'Decay',
    'k_LiP',
    'lam_ne',
    'lam_pe',
    'kcr_ne',
    'Vsei',
    "Eact_SEI", # 1e4~3.8e4
    "Eact_kcr_ne",#0
    "Eact_Ds_ne",#1.7e4
    "Eact_Ds_pe",#1.2e4
    #"Rct",#0.010
    #'h',#20
    'Elely res', # 1.0 or 0.99
]
# Dint_default = [5e-18,];    Rct_default=[0.010,];h_default = [20,];
# Eact_SEI_default=[1e4,]; V_SEI_default = [6.67e-5]  Eact_Ds_Ne_default=[1.7e4,]
Decay_default = [1e-7,]
k_LiP_default = [1e-10,]   
LAM_Pe_default = [2.98e-18,]  
LAM_Ne_default = [2.84e-9,]
 
k_cr_Ne_default=[5.29e-25]


Eact_cr_Ne_default=[0.0,]  

Eact_Ds_Pe_default=[1.2e4,]

Elely_res_default = [1.0,]


In [3]:
# old functions
def save_rows_to_csv(file_path, rows, header):
    with open(file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)  # Write parameter names as the header row
        writer.writerows(rows)
def generate_combinations(Bounds, Num_tot):
    lower_bounds = []
    upper_bounds = []
    for bound in Bounds:
        lower_bounds.append(bound[0])
        upper_bounds.append(bound[1])
    combinations = []
    for _ in range(Num_tot):
        combination = []
        for lower, upper in zip(lower_bounds, upper_bounds):
            value = random.uniform(lower, upper)
            combination.append(value)
        combinations.append(combination)
    return combinations

def save_combinations_to_csv(combinations, parameter_names, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(parameter_names)  # Write parameter names as the first row
        for combination in combinations:
            writer.writerow(combination)

In [4]:
BasicPath =  os.path.expanduser("~/EnvPBGEM_NC/SimSave")
Target  = f'/InputData/'
if not os.path.exists(BasicPath + Target):
   os.mkdir(BasicPath + Target)

In [5]:
def Get_Scan_files(
        BasicPath,Target_name,model_options,
        parameter_names,para_short_name,
        Pack,   num,  i_base,
        rows_per_file,Bundle):
    
    import itertools
    para_dict_Same = {
        "Cycles within RPT":1,
        "RPT temperature":25,
        
        "Para_Set": "OKane2023",
        "Model option":model_options,
        
        }
    unchange_key2 = list(para_dict_Same.keys())
    unchange_val2 = list(para_dict_Same.values())
    short_pack = [lst for lst in Pack if len(lst) > 1]
    selected_indices = [i for i, lst in enumerate(Pack) if len(lst) > 1]
    shortList_para_short_name = [para_short_name[i] for i in selected_indices]
    shortList_para_short_name.insert(0,"No")
    really_change_val =  [
        list(comb) for comb in itertools.product(*short_pack)]

    change_val =  [
        list(comb) for comb in itertools.product(*Pack)]
    combinations = [[i+1,*elem, *unchange_val2] for i,elem in enumerate(change_val)]
    comb_short   = [[i+1,*elem] for i,elem in enumerate(really_change_val)]
    parameter_names = [*parameter_names,*unchange_key2]
    print("Total cases number is",len(combinations))
    if Bundle:
        # Specify the total number of cases
        total_cases = len(combinations)
        # Specify the number of rows per CSV file, rows_per_file
        # Calculate the number of files needed
        num_files = (total_cases - 1) // rows_per_file + 1
        # Create the target folder
        folder_path = os.path.join(BasicPath, "InputData", Target_name)
        os.makedirs(folder_path, exist_ok=True)
        # Write data to each CSV file
        for i in range(num_files):
            file_name = f"Bundle_{i+1+i_base}.csv"
            file_path = os.path.join(folder_path, file_name)
            start_row = i * rows_per_file
            end_row = min(start_row + rows_per_file, total_cases)
            rows = combinations[start_row:end_row]
            save_rows_to_csv(file_path, rows, parameter_names)
        filename = BasicPath+f"/InputData/{Target_name}/"+f'{Target_name}_i_base_{i_base}.csv'
        filename_short = BasicPath+f"/InputData/{Target_name}/"+f'{Target_name}_i_base_{i_base}_s.csv'
        save_combinations_to_csv(combinations, parameter_names, filename)
        save_combinations_to_csv(comb_short, shortList_para_short_name, filename_short)
        print(f"Combinations saved to '{Target_name}.csv' file.") 
        print(f"CSV files created in folder '{Target_name}'.")
    else:
        filename = BasicPath+"/InputData/"+f'{Target_name}.csv'
        save_combinations_to_csv(combinations, parameter_names, filename)
        print(f"Combinations saved to '{Target_name}.csv' file.") 
    return len(combinations)


In [6]:
def get_list_from_tuple(d, num):
    if d[1] > d[0]:
        if d[1] > 100 * d[0]:
            result_list = (np.exp(np.linspace(np.log(d[0]), np.log(d[1]), num=num))).tolist()
        else:
            result_list = (np.linspace(d[0], d[1], num=num)).tolist()
    else:
        result_list = []
    return result_list
def Get_Scan_Orth_Latin(
        BasicPath,Target_name,model_options,
        parameter_names,para_short_name,
        Pack, num,
        rows_per_file,Bundle):
    
    import itertools; from pyDOE import lhs
    para_dict_Same = {
        "Cycles within RPT":1,
        "RPT temperature":25,
        #"Mesh list":[5,5,5,60,20],  
        "Para_Set": "OKane2023",
        "Model option":model_options,
        "Current solvent concentration in the reservoir [mol.m-3]":4541.0,
        "Current electrolyte concentration in the reservoir [mol.m-3]":1000,
        "Ratio of Li-ion concentration change in " 
        "electrolyte consider solvent consumption":1.0,
        'EC initial concentration in electrolyte [mol.m-3]':4541.0,
        'Typical EC concentration in electrolyte [mol.m-3]':4541.0, 
        }
    unchange_key2 = list(para_dict_Same.keys())
    unchange_val2 = list(para_dict_Same.values())
    
    Pack_tuple = []; Pack_tuple_index = []
    Pack_list = [];  Pack_list_index  = []
    for i,item in enumerate(Pack):
        if isinstance(item, tuple):
            Pack_tuple.append(item)
            Pack_tuple_index.append(i)
        elif isinstance(item, list):
            Pack_list.append(item)
            Pack_list_index.append(i)
    com_tuple = []; comb_tu_list =[]
    if len(Pack_tuple) > 1:
        for tuple_i in Pack_tuple:
            com_tuple.append( get_list_from_tuple(tuple_i, num) )
        # apply Latin Hypercube:
        #print(com_tuple)
        samples = lhs(len(com_tuple), samples=num)
        for sample in samples:
            combination = []
            for i, candidate_list in enumerate(com_tuple):
                index = int(sample[i] * num)
                combination.append(candidate_list[index])
            comb_tu_list.append(combination)
    else:
        print("error! Pack_tuple must has 2 elements")
    # apply product sampling:
    comb_li_list = [list(comb) for comb in itertools.product(*Pack_list)]
    #print(comb_tu_list)
    #print(comb_li_list)
    Big_Comb = []
    for comb_tu in comb_tu_list:
        for comb_li in comb_li_list:
            big_comb = [0] * (len(comb_tu)+len(comb_li))
            for comb_tu_i,index in zip(comb_tu,Pack_tuple_index):
                big_comb[index] = comb_tu_i
            for comb_li_i,index in zip(comb_li,Pack_list_index):
                big_comb[index] = comb_li_i
            Big_Comb.append(big_comb)
    #print(Big_Comb)
    Big_Comb
    combinations = [[i+1,*elem, *unchange_val2] for i,elem in enumerate(Big_Comb)]

    parameter_names = [*parameter_names,*unchange_key2]
    print("Total cases number is",len(combinations))
    if Bundle:
        # Specify the total number of cases
        total_cases = len(combinations)
        # Specify the number of rows per CSV file, rows_per_file
        # Calculate the number of files needed
        num_files = (total_cases - 1) // rows_per_file + 1
        # Create the target folder
        folder_path = os.path.join(BasicPath, "InputData", Target_name)
        os.makedirs(folder_path, exist_ok=True)
        # Write data to each CSV file
        for i in range(num_files):
            file_name = f"Bundle_{i+1}.csv"
            file_path = os.path.join(folder_path, file_name)
            start_row = i * rows_per_file
            end_row = min(start_row + rows_per_file, total_cases)
            rows = combinations[start_row:end_row]
            save_rows_to_csv(file_path, rows, parameter_names)
        filename = BasicPath+f"/InputData/{Target_name}/"+f'{Target_name}.csv'
        # filename_short = BasicPath+f"/InputData/{Target_name}/"+f'{Target_name}_s.csv'
        save_combinations_to_csv(combinations, parameter_names, filename)
        # save_combinations_to_csv(comb_short, shortList_para_short_name, filename_short)
        print(f"Combinations saved to '{Target_name}.csv' file.") 
        print(f"CSV files created in folder '{Target_name}'.")
    else:
        filename = BasicPath+"/InputData/"+f'{Target_name}.csv'
        save_combinations_to_csv(combinations, parameter_names, filename)
        print(f"Combinations saved to '{Target_name}.csv' file.") 
    return len(combinations)


# SEI only

In [51]:
options_SEI = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",
    
    "lithium plating": "none",
    "lithium plating porosity change":"false",
    "particle mechanics": "constant cracks",
    "loss of active material": "none",

    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
Exps  = [2]
Temps = [10,25,40]
Mesh_list = [ "[10,5,5,100,20]",]

cands_Dint  =   [2.3555555e-18] # (np.logspace(np.log10(1e-20), np.log10(1e-17), num=10)).tolist() #
cands_V_SEI =   [4e-5,] # (np.linspace(3e-05, 12e-05,10)).tolist() #
cands_Eact_SEI=  [ 1e4 ] # (np.logspace(np.log10(1e1), np.log10(1e5), num=10)).tolist() #
cands_Eact_Ds_Ne = (np.linspace(2e4, 6e4,10)).tolist() #  [ 6e4, ] # 

cands_Elely_res = [0.99,]

Pack = [
    Exps,Temps,Mesh_list,
    cands_Dint,    
    Decay_default, k_LiP_default,LAM_Ne_default,
    LAM_Pe_default,k_cr_Ne_default,
    cands_V_SEI,
    cands_Eact_SEI,Eact_cr_Ne_default,
    cands_Eact_Ds_Ne,Eact_Ds_Pe_default,
    cands_Elely_res,
]
Target_name = "SEI_Exp2_NC_SA"
rows_per_file = 1; 
num = 20; i_base = 30 * 3
num_scans = Get_Scan_files(
    BasicPath,Target_name,options_SEI,
    parameter_names,para_short_name,
    Pack, num, i_base,
    rows_per_file,Bundle=True)

Total cases number is 30
Combinations saved to 'SEI_Exp2_NC_SA.csv' file.
CSV files created in folder 'SEI_Exp2_NC_SA'.


# SEI_Dry

In [57]:
options_SEI_Dry = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",
    
    "lithium plating": "none",
    "lithium plating porosity change":"false",
    "particle mechanics": "constant cracks",
    "loss of active material": "none",

    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
Exps  = [2]
Temps = [10,25,40]
Mesh_list = [ "[10,5,5,100,20]",]

cands_Dint  = [1.2e-18]   #  (np.logspace(np.log10(1e-20), np.log10(1e-17), num=10)).tolist() #
cands_V_SEI = [6.74e-5]# (np.linspace(3e-05, 12e-05,10)).tolist() # 

cands_Eact_SEI=  [  1e4 ] # (np.logspace(np.log10(1e1), np.log10(1e5), num=10)).tolist() #
cands_Eact_Ds_Ne = (np.linspace(2e4, 6e4,10)).tolist() # [ 6e4, ]# 

cands_Elely_res = [1.0,]

Pack = [
    Exps,Temps,Mesh_list,
    cands_Dint,    
    Decay_default, k_LiP_default,LAM_Ne_default,
    LAM_Pe_default,k_cr_Ne_default,
    cands_V_SEI,
    cands_Eact_SEI,Eact_cr_Ne_default,
    cands_Eact_Ds_Ne,Eact_Ds_Pe_default,
    cands_Elely_res,
]
Target_name = "SEI_Dry_Exp2_NC_SA"
rows_per_file = 1;  num = 100; i_base = 30 * 3
num_scans = Get_Scan_files(
    BasicPath,Target_name,options_SEI_Dry,
    parameter_names,para_short_name,
    Pack, num,i_base,
    rows_per_file,Bundle=True)


Total cases number is 30
Combinations saved to 'SEI_Dry_Exp2_NC_SA.csv' file.
CSV files created in folder 'SEI_Dry_Exp2_NC_SA'.


# Full

In [ ]:
(9.39-8)*13800

In [7]:

options_Full = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",
    "lithium plating": "partially reversible",
    "lithium plating porosity change":"true",
    "particle mechanics": ("swelling and cracking", "swelling only"),
    "loss of active material": "stress-driven",
    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
Exps  = [2]
Temps = [10,25,40]
Mesh_list = [ "[10,5,5,100,20]",]

cands_Dint  =   [9.81e-19,] # (np.logspace(np.log10(1e-20), np.log10(1e-17), num=10)).tolist() # 
cands_V_SEI =   [5.22e-5,] #  (np.linspace(3e-05, 12e-05,10)).tolist() # 
cands_Eact_SEI= [ 5e3, ] # (np.logspace(np.log10(1e1), np.log10(1e5), num=10)).tolist() # 
cands_Decay =    [1e-7]    #  (np.linspace(2.5e-8, 2.5e-6,10)).tolist() # 
cands_LiP =     [1e-10]    #  (np.linspace(2.5e-11, 2.5e-9,10)).tolist() #  

cands_LAM_Pe =   [2.98e-18] # (np.logspace(np.log10(1e-18), np.log10(1e-8), num=10)).tolist() #

cands_LAM_Ne =  [2.84e-9]  # (np.logspace(np.log10(1e-18), np.log10(1e-8), num=10)).tolist() # 
cands_k_cr_Ne=  [5.29e-25] # (np.logspace(np.log10(1e-25), np.log10(1e-20), num=10)).tolist() # 

cands_Eact_Ds_Ne= (np.linspace(2e4, 6e4,10)).tolist() #  [ 2e4,] #  

cands_Elely_res = [1.0,]

Pack = [
    Exps,Temps,Mesh_list,
    cands_Dint,    
    cands_Decay, cands_LiP,cands_LAM_Ne,
    cands_LAM_Pe,cands_k_cr_Ne,
    cands_V_SEI,
    cands_Eact_SEI,Eact_cr_Ne_default,
    cands_Eact_Ds_Ne,Eact_Ds_Pe_default,
    cands_Elely_res,
]
Target_name = "5_coupled_demo"
rows_per_file = 1;   num = 60;i_base = 30 * 8
num_scans = Get_Scan_files(
    BasicPath,Target_name,options_Full,
    parameter_names,para_short_name,
    Pack,  num,i_base,
    rows_per_file,Bundle=True)


Total cases number is 30
Combinations saved to '5_coupled_demo.csv' file.
CSV files created in folder '5_coupled_demo'.
